In [3]:
import html
import os
import json
import random
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
from bs4 import BeautifulSoup

# Set up Selenium WebDriver
driver_path = '/Users/harshpreet/downloads/chromedriver-mac-arm64/chromedriver'
service = Service(driver_path)
driver = webdriver.Chrome(service=service)

# Prepare output directories
base_dir = "./data/anonymized_editorials"
text_output_dir = os.path.join(base_dir, "data_chunks")
metadata_output_dir = os.path.join(base_dir, "meta_files")
os.makedirs(text_output_dir, exist_ok=True)
os.makedirs(metadata_output_dir, exist_ok=True)

def randomize_text(text):
    """Add some noise to text for obfuscation."""
    words = text.split()
    for i in range(len(words)):
        if random.random() > 0.7:  # Random chance to replace a word
            words[i] = words[i][::-1]  # Reverse the word
    return " ".join(words)

try:
    # Open the target URL
    url = "https://codeforces.com/blog/entry/128716"
    driver.get(url)
    
    # Wait for the page to load dynamically
    WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.TAG_NAME, "body")))

    # Find the specific href section
    target_href = "/contest/2000/problem/A"
    target_element = driver.find_element(By.XPATH, f"//a[@href='{target_href}']")
    if target_element:
        # Random delay to avoid detection
        time.sleep(random.uniform(2, 5))

        # Scroll to the element
        ActionChains(driver).move_to_element(target_element).perform()

        # Extract the surrounding content
        parent_div = target_element.find_element(By.XPATH, "./ancestor::div")
        soup = BeautifulSoup(parent_div.get_attribute("innerHTML"), 'html.parser')

        # Initialize a dictionary to store extracted data
        extracted_data = {}

        # Extract target segments with anonymized names
        segment_titles = ["Segment A", "Segment B", "Segment C", "Segment D"]
        search_patterns = ["Hint 1", "Solution", "Rate this problem", "C++ Code"]
        for title, pattern in zip(segment_titles, search_patterns):
            section_title = soup.find("b", string=pattern)
            if section_title:
                section_content = section_title.find_next("div", class_="spoiler-content") or section_title.find_next("pre")
                extracted_data[title] = randomize_text(section_content.text.strip()) if section_content else f"No content for {title}."

        # Save extracted content
        sanitized_problem_id = target_href.split("/")[-1].replace("/", "_")
        random_id = f"{sanitized_problem_id}_{random.randint(1000, 9999)}"
        text_filename = os.path.join(text_output_dir, f"chunk_{random_id}.txt")
        metadata_filename = os.path.join(metadata_output_dir, f"meta_{random_id}.json")

        # Save text data
        with open(text_filename, "w", encoding="utf-8") as text_file:
            for key, value in extracted_data.items():
                text_file.write(f"{key}:\n{value}\n\n")

        # Save metadata as JSON
        with open(metadata_filename, "w", encoding="utf-8") as metadata_file:
            json.dump(extracted_data, metadata_file, indent=4)

    else:
        print(f"No section found for href='{target_href}'.")

finally:
    # Close the driver
    driver.quit()


NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//a[@href='/contest/2000/problem/A']"}
  (Session info: chrome=131.0.6778.205); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
0   chromedriver                        0x0000000104e7b184 cxxbridge1$str$ptr + 3626716
1   chromedriver                        0x0000000104e739d4 cxxbridge1$str$ptr + 3596076
2   chromedriver                        0x00000001048e0968 cxxbridge1$string$len + 89228
3   chromedriver                        0x0000000104924d50 cxxbridge1$string$len + 368756
4   chromedriver                        0x000000010495e5b4 cxxbridge1$string$len + 604376
5   chromedriver                        0x0000000104919568 cxxbridge1$string$len + 321676
6   chromedriver                        0x000000010491a1b8 cxxbridge1$string$len + 324828
7   chromedriver                